# Resumo do que fizemos em IA até agora, somente aplicações práticas

## Exemplo de embedding


In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mistral",base_url='')

len(embeddings.embed_query("Hello, world!"))

## Exemplo báisco de Tool Calling


In [ ]:
from langchain_ollama import ChatOllama

model_tool = ChatOllama(model='llama3.2',base_url='',temperature=0)
def add(a, b):
    return a + b
def sub(a, b):
    return a - b
def mul(a, b):
    return a * b
def div(a, b):
    return a / b

tools = [add, sub, mul, div]
model_tool = model_tool.bind_tools(tools)
query = "What is 3 + 2? "
response = model_tool.invoke(query).tool_calls

for call in response:
    name,args = call['name'], call['args']
    result = eval(name)(**args)
    print(result)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


def evaluate_operation(operation):
    name = operation['name']
    parameters = operation['parameters']
    
    # Recursively evaluate any nested operations in the parameters
    for key, value in parameters.items():
        if isinstance(value, dict) and 'name' in value and 'parameters' in value:
            parameters[key] = evaluate_operation(value)
    
    # Evaluate the operation with the processed parameters
    return eval(name)(**parameters)


query2 = "What is 3 + 2? Take the result and multiply by 2"

messages = [SystemMessage('''
If you need to apply a funtion to the result of another one, you should do as such: {'name':'function1','parameters':{'a':1,'b':{'name':'function2','parameters':{'a':1,'b':2}}}}'''),HumanMessage(query2)]

response = model_tool.invoke(messages)
messages.append(AIMessage(str(response.content)))
package = response.content.split(';')
print(package)




for x in package:
    operation = eval(x)  # Convert JSON string to dictionary
    result = evaluate_operation(operation)
    print(result)
    #messages.append(AIMessage(str(result)))
    


## Exemplo de processamento de Imagem

In [ ]:
import ollama
client = ollama.Client(host='')

response = client.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image?',
        'images': ['plano_de_fundo.jpg']
    }]
)
print(response.message.content)

## Exemplo de RAG com chroamDB
Estamos migrando para milvusDB

In [ ]:
chroma_client = chromadb.HttpClient(host='')
print(chroma_client.list_collections())
collection = chroma_client.get_collection('genie_normal')
collection.query(
    query_texts=['SSID'],
    n_results=4,
    include=['documents']
)['documents'][0]

#usamos a llama localmente para para embeddings, 5a feira eu mostro

## Demonstração Principal

In [ ]:
import genieacs
def change(parameter:str, value:str):
    device_id = '5091E3-EX141-2237011003026'
    acs = genieacs.Connection
    print(f"Changing parameter {parameter} to {value}")
    acs.task_set_parameter_values(device_id, [[parameter, value]])
    print (f"Parameter {parameter} changed to {value}")

def get(parameter:str):
    device_id = '5091E3-EX141-2237011003026'
    acs = genieacs.Connection
    acs.device_get_parameter(device_id, parameter)
    print(f"The parameter {parameter} is {acs.device_get_parameter(device_id, parameter)}")


# adatação de PROMPT
query = 'Change the name of the wifi network to "MyWifi"'
collection = chroma_client.get_collection('genie_normal')
text = collection.query(
    query_texts=[query],
    n_results=4,
    include=['documents']
)['documents'][0]
print(text)


#Passa contexto para o modelo
system_message = SystemMessage(content=f'Here are your instructuions: {text}')
human_message = HumanMessage(content=query)
#print(llm.invoke([system_message,human_message]).content)

#Chama a função
tools = [change,get]
llm_tool = llm.bind_tools(tools)
response = llm_tool.invoke([system_message,human_message])
print(response)
package = response.content.split(';')

#Interpreta a Funçaõ
for x in package:
    x= eval(x)
    name , param = x['name'], x['parameters']
    result = eval(name)(**param)
